## 1.Загрузим данные

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
text_PS = pd.read_csv('positive.csv',sep=';',error_bad_lines=False, header=None).iloc[:,[3,4]]
text_NG = pd.read_csv('negative.csv',sep=';',error_bad_lines=False, header=None).iloc[:,[3,4]]

In [3]:
text_PS[4]=1 # positive
text_NG[4]=0 # negative

In [4]:
# проверим на сбалансированность
print(text_PS.shape,text_NG.shape)

(114911, 2) (111923, 2)


In [5]:
# обьеденим для удобства работы
data=pd.concat([text_PS,text_NG])

## Обработаем данные

In [6]:
import re as re

In [7]:
# Уберем теги @ и RE
data[3] = data[3].map(lambda x:re.sub(r'[R][T]\s+[@]\w+[:]\s*|[@]\w+\s+','', x))
# заменим ссылки на SH_T_T_P
# заменим цифры на D_G_TII
data[3] = data[3].map(lambda x:re.sub(r'\d\w*','d_g_t', x))

In [8]:
print(data.head(10))

                                                   3  4
0  хоть я и школота, но поверь, у нас то же самое...  1
1  Да, все-таки он немного похож на него. Но мой ...  1
2             Ну ты идиотка) я испугалась за тебя!!!  1
3  "Кто то в углу сидит и погибает от голода, а м...  1
4  Вот что значит страшилка :D\nНо блин,посмотрев...  1
5  ну любишь или нет? — Я не знаю кто ты бля:D ht...  1
6  Ох,d_g_t :D ну это конечно же . Чтобы у нее бы...  1
7  У тебя есть ухажёр? Нет - мои уши не кто не жр...  1
8            Поприветствуем моего нового читателя ;)  1
9  Теперь у меня есть частичка Сиднея :) #Sydney ...  1


## CountVectorizer

In [9]:
def my_tokenizer (inp):
    return re.split(r'[;?!,\s]',inp)

In [14]:
import sklearn.feature_extraction.text
vctr = sklearn.feature_extraction.text.CountVectorizer(min_df=10,max_df=0.8,tokenizer=my_tokenizer)
#vctr = sklearn.feature_extraction.text.CountVectorizer(tokenizer=my_tokenizer)
# нужен свой шаблон token_pattern=
# min_df=5 слово должно встречаться минимум в 5 докумнтах
# max_df=0.9 уберем слова которые встречаются более чем в 90 процентах докуметов

In [15]:
data_vect = vctr.fit_transform(data[3])

In [16]:
data_vect.shape

(226834, 17797)

In [17]:
#vctr.vocabulary_

In [18]:
vctr.vocabulary_.get(':-)')

506

## TF.IDF vectorizer

In [19]:
vctr_tfidf = sklearn.feature_extraction.text.TfidfTransformer()
data_tfidf = vctr_tfidf.fit_transform(data_vect)

In [20]:
data_tfidf.shape

(226834, 17797)

In [21]:
del(data_vect)

## Разделим выборки

In [22]:
# разделим на тестовую и валидационную
y=data[4]
del data[4]
X_train, X_test, y_train, y_test = train_test_split(data_tfidf, y, test_size=0.33, random_state=442)

In [23]:
X_train.shape

(151978, 17797)

## Классифицируем

In [ ]:
# Баес
import sklearn.naive_bayes
bb=sklearn.naive_bayes.MultinomialNB(alpha=5)
bb.fit(X_train, y_train)
predicted=bb.predict(X_test)

In [28]:
%%time
#SVM
from sklearn.svm import SVC
svm=SVC(kernel="linear", verbose=True)
svm.fit(X_train, y_train)
predicted=svm.predict(X_test)

[LibSVM]Wall time: 1h 50min 42s


## Оценим качество

In [29]:
import sklearn.metrics

In [30]:
sklearn.metrics.accuracy_score(y_test, predicted)

0.50721385059313884

In [ ]:
sklearn.metrics.confusion_matrix(y_test, predicted)

## Результаты Accuracy

#Баес
0.7476 - чистый
0.7390 - убрал хештеги и числа
0.7380 + CountVectorizer(min_df=5,max_df=0.9)
0.8387 + токенайзер который пропускает варианты с :-) и :-( c пробелом
0.8452 + токенайзер который пропускает варианты с :-) и :-( c пробелом и тд
0.8617 + токенайзер который пропускает варианты с :-) и :-( c r'[;?!,\s]'
0.8457 - только свой токенайзер 
--- чем болше данных тем лучше работает

#SVM
0.8227 linear C=0.025 \хештеги и числа и min_df=5,max_df=0.9
0.8569 linear C=1 \хештеги и числа и min_df=10,max_df=0. //55 min ФИНАЛ
0.50 rbf C=1 \хештеги и числа и min_df=10,max_df=0. //1:55 min
--- явно лучше класифицирует на меньшем обьеме фич лучший кандидат на финальный тюниг
